# Raw data
--------------
## first check the raw data

In [4]:
# check how many lines
! wc -l data/emojitweets-01-04-2018.txt

18866900 data/emojitweets-01-04-2018.txt


In [2]:
# check how many words
! wc -w data/emojitweets-01-04-2018.txt

209430000 data/emojitweets-01-04-2018.txt


In [3]:
# On average the sentense are have 11 words
209430000/18866900

11.100392751326398

## Train - test - validation split

In [6]:
# split on 80-10-10
18866900 * 0.8, 18866900 * 0.1, 18866900 * 0.2

(15093520.0, 1886690.0, 3773380.0)

In [8]:
! head -15093520 data/emojitweets-01-04-2018.txt >>emoji_train.txt


In [14]:
! (tail -3773380 data/emojitweets-01-04-2018.txt | head -1886690) >> emoji_val.txt

tail: error writing 'standard output': Broken pipe


In [10]:
! tail -1886690 data/emojitweets-01-04-2018.txt >> emoji_test.txt

In [1]:
# generate small_sample
! head -1509352 data/emoji_train.txt >> data/emoji_ss.txt

# Word Vectorize

In [26]:
# Python program to generate word vectors using Word2Vec 
  
# importing all necessary modules 
from nltk.tokenize import sent_tokenize, word_tokenize 
import warnings 
  
warnings.filterwarnings(action = 'ignore') 
  
import gensim 
from gensim.models import Word2Vec 
  

In [1]:
# sample = open("emoji_1M.txt", "r") 
sample = open("emoji_train.txt", "r") 
s = sample.read() 
  
# Replaces escape character with space 
f = s.replace("\n", " ") 
  
data = [] 
  

In [27]:
import emoji
import regex

def split_count(text):

    emoji_list = []
    data = regex.findall(r'\X', text)
    for word in data:
        if any(char in emoji.UNICODE_EMOJI for char in word):
            emoji_list.append(word)

    return emoji_list

In [124]:
# 
sample = open("../emoji_100k.txt", "r") 
s_ss = sample.read() 
  
# Replaces escape character with space
f_ss = s_ss.lower().replace("\n", " ")

# replace all 
signs = [",", ".", "(" ,")", "'", '/"', ":", ";", "/", "(", ")"]
for sign in signs:
    f_ss = f_ss.replace(sign, "")
  
data = [] 

In [125]:
f = f_ss
# iterate through each sentence in the file 
for i in sent_tokenize(f): 
    temp = [] 
      
    # tokenize the sentence into words 
    for j in word_tokenize(i): 
        temp.append(j.lower()) 
  
    data.append(temp) 

In [126]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import pos_tag
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [127]:
# import stopwords
stop = set(stopwords.words('english'))



In [35]:
# remove stop words and snowball stemmer
out = [[word for word in words if word not in stop] for words in data]
snowball = SnowballStemmer('english')
docs_snowball = [[snowball.stem(word) for word in words] for words in out]

In [128]:
docs_snowball[0]

['squad',
 'arriv',
 'game',
 '2',
 '🚀',
 'dude',
 'like',
 '5',
 '’',
 '8',
 '140',
 'pound',
 'dick',
 'long',
 'strong',
 'alway',
 'littl',
 'dude',
 'carri',
 '🍆',
 '\U0001f92a🙃',
 'followers👇',
 'cant',
 'breatiuhw',
 '💀💀💀',
 '2️⃣4️⃣',
 'hour',
 'til',
 'schedul',
 'drop',
 '!']

save emoji list

In [10]:
# emoji_list = list(emoji_set)
# with open("emoji_list.csv", 'w', newline='') as myfile:
#      w = csv.writer(myfile, quoting=csv.QUOTE_ALL)
#      w.writerow(emoji_list)

4358

In [32]:
with open("emoji_list.csv", 'r', newline='') as myfile:
     emoji_list = csv.reader(myfile, quoting=csv.QUOTE_ALL)

FileNotFoundError: [Errno 2] No such file or directory: 'emoji_list.csv'

# First model

In [38]:
# Create CBOW model 
model1 = gensim.models.Word2Vec(data, min_count = 1,  
                              size = 100, window = 5) 

KeyboardInterrupt: 

In [15]:
# Save CBOW model
import pickle
with open('cbow_ss.pkl', 'wb') as f:
    pickle.dump(model1, f)

In [18]:
model1.estimate_memory()

{'vocab': 1606239500,
 'vectors': 1284991600,
 'syn1neg': 1284991600,
 'total': 4176222700}

In [27]:
emoji_list = list(emoji_set)

In [45]:
def emoji_prodictor(word):
    sim = model1.most_similar(word,topn=50)
    for i in sim:
        if i[0] in emoji_list:
            return i[0]
    return word

In [47]:
s1 = "let's go grocery today to get some bike related stuff"
for w in s1.lower().split(" "):
    print(emoji_prodictor(w))


let's
go
grocery
today
🤷🏽‍♀️
get
😂
🚲
related
stuff


In [44]:
emoji_prodictor("bike")

'bike'

In [35]:
emoji_set = set(split_count(f))


In [41]:
emoji_set = list(emoji_set)

In [129]:
# Create CBOW model 
model2 = gensim.models.Word2Vec(docs_snowball, 
                                min_count = 5,  
                                size = 100, 
                                window = 5,
                                workers=4) 

In [41]:
# Save CBOW model
import pickle
with open('cbow_ss.pkl', 'wb') as f:
    pickle.dump(model2, f)

In [130]:
def emoji_prodictor(word):
    sim = model2.most_similar(word,topn=20)
    for i in sim:
        if i[0] in emoji_set:
            return i[0]
    return word

In [131]:
def word_pipeline(sentense):
    out = [word for word in sentense.lower().split(" ") if word not in stop]
    out = [snowball.stem(word) for word in out]
    return out
    
    
s1 = "let's go grocery today to get some bike related stuff"
word_pipeline(s1)

['let', 'go', 'groceri', 'today', 'get', 'bike', 'relat', 'stuff']

In [132]:
s1 = "let's go grocery today to get some bike related stuff"
for w in word_pipeline(s1):
    print(emoji_prodictor(w))


🤷‍♀️
🤷🏼‍♀️
groceri
😨
🤷🏾‍♀️
🚘
relat
stuff


In [133]:
model2.most_similar("bike",topn=20)

[('car', 0.6086397767066956),
 ('🚘', 0.5864426493644714),
 ('wheel', 0.5853496193885803),
 ('cruis', 0.5828701853752136),
 ('highway', 0.5649341940879822),
 ('hors', 0.552920937538147),
 ('driveway', 0.5436760783195496),
 ('kitchen', 0.5423159003257751),
 ('bus', 0.5340845584869385),
 ('road', 0.5305561423301697),
 ('hike', 0.5252187848091125),
 ('crib', 0.52373206615448),
 ('tabl', 0.5200468301773071),
 ('edg', 0.5186354517936707),
 ('stair', 0.517928957939148),
 ('truck', 0.5167300701141357),
 ('desert', 0.5161859393119812),
 ('beach', 0.5118310451507568),
 ('around', 0.5103741884231567),
 ('dread', 0.509566068649292)]

-----------------
validation set

In [1]:
val_df = pd.read_csv("data/emoji_val.2csv",index_col=0)

In [46]:
emoji_set = val_df["emoji"].values.tolist()

In [4]:
val_df["full_desc"] = val_df.full_desc.apply(lambda x: x.strip('][').split(', '))
# val_df["full_desc"] = val_df.full_desc.apply(lambda x: x.strip('][').split(', '))

In [18]:
for i in val_df.index:
    for j in range(len(val_df.full_desc[i])):
#         print(i)
        val_df.full_desc[i][j] = val_df.full_desc[i][j].strip("/'")

In [83]:
# build validation set
val_dict = {}
lst = []
for i in val_df.index.tolist():
    lst.extend(val_df.full_desc[i])
    

In [85]:
vals = list(set(lst))
for val in vals:
    val_dict[val] = []
    for index in val_df.index:
        
        if val in val_df.full_desc[index]:
            val_dict[val].append(val_df.emoji[index])


In [138]:
def word2vec_score(val_dict, model):
    total = 0
    correct = 0
    words = []
    word_vectors = model.wv
    for word in val_dict:
#         print(word)
        if word_pipeline(word):
            w = word_pipeline(word)[0]
            words.append(w)
            if w in word_vectors:
                total += 0
                if emoji_prodictor(w) in val_dict[word]:
                    correct += 1
    return correct, total, words
            

In [139]:
word2vec_score(val_dict, model2)

(163,
 0,
 ['reliev',
  'five',
  'lacross',
  'twelv',
  'merman',
  '3',
  'massag',
  'penguin',
  'luxembourg',
  'merperson:',
  'speak-no-evil',
  'stopwatch',
  'eight-thirti',
  'crossbon',
  'cake',
  'drool',
  'take',
  'frame',
  'ecuador',
  'ten-thirti',
  'atom',
  'artist:',
  'avocado',
  'mammoth',
  'handbag',
  'bug',
  'mantelpiec',
  'taco',
  'angel:',
  'mechan',
  'arabia',
  'ear',
  'floppi',
  'spring',
  'exchang',
  'ghost',
  'ok',
  'splay',
  'lion',
  'straw',
  'christma',
  'motor',
  'wood',
  'uganda',
  'melon',
  'hug',
  'sake',
  'traffic',
  'brunei',
  'tractor',
  'spaghetti',
  'volum',
  'roll',
  'transgend',
  '5',
  'hear',
  'tomato',
  'wolf',
  'bolivia',
  'timer',
  '2',
  'helicopt',
  'desert',
  'packag',
  'haircut',
  'hockey',
  'koala',
  'sos',
  'castl',
  'polynesia',
  'príncipe',
  'movi',
  'slider',
  'thailand',
  'needl',
  'niue',
  'tropic',
  'wing',
  '“bargain”',
  'trident',
  'curl',
  'cupcak',
  'claim',
  

In [145]:
model2.most_similar('1',topn=20)

[('2', 0.8036147356033325),
 ('3', 0.7338453531265259),
 ('4', 0.5989294052124023),
 ('8', 0.5632615089416504),
 ('13', 0.5582780838012695),
 ('6', 0.5528515577316284),
 ('10', 0.5463414788246155),
 ('12', 0.5445541739463806),
 ('per', 0.521578848361969),
 ('9', 0.5070649981498718),
 ('7', 0.500156044960022),
 ('48', 0.4976421296596527),
 ('800', 0.4964720606803894),
 ('5', 0.49304234981536865),
 ('36', 0.4899243712425232),
 ('4566', 0.4803587794303894),
 ('1…', 0.4803035259246826),
 ('11', 0.4702182412147522),
 ('📣1,000', 0.46588796377182007),
 ('1/2', 0.4515560567378998)]

In [ ]:
def word_pipeline(sentense):
    out = [word for word in sentense.lower().split(" ") if word not in stop]
    out = [snowball.stem(word) for word in out]
    return out

In [115]:
sentense = "under"
[word for word in sentense.lower().split(" ") if word not in stop]

[]

In [116]:
word_pipeline(sentense)

[]